In [1]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim

In [2]:
batch_size = 100
epochs = 25
num_classes=10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
data_transforms = torchvision.transforms.Compose([torchvision.transforms.ToTensor()])

train_set = torchvision.datasets.MNIST('.data/', train=True, download=True, transform=data_transforms)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, shuffle=True)

test_set = torchvision.datasets.MNIST('.data/', train=False, download=True, transform=data_transforms)
test_loader = torch.utils.data.DataLoader(test_set,batch_size=batch_size, shuffle=True)

In [4]:
x_batch, y_batch = iter(train_loader).next()
print("Training set: {} samples - Max value: {} - Min value: {}".format(len(train_loader.dataset), 
                                                                        x_batch.max(), x_batch.min()))

Training set: 60000 samples - Max value: 1.0 - Min value: 0.0


In [5]:
x_batch, y_batch = iter(test_loader).next()
print("Test set: {} samples - Max value: {} - Min value: {}".format(len(test_loader.dataset), 
                                                                        x_batch.max(), x_batch.min()))

Test set: 10000 samples - Max value: 1.0 - Min value: 0.0


In [6]:
print("Example batch shape: {}".format(x_batch.shape))

Example batch shape: torch.Size([100, 1, 28, 28])


In [7]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.linear1 = nn.Linear(784, 1024)
        self.relu1 = nn.ReLU()
        self.linear2 = nn.Linear(1024, 1024)
        self.relu2 = nn.ReLU()
        self.linear3 = nn.Linear(1024, 1024)
        self.relu3 = nn.ReLU()
        self.classifier = nn.Linear(1024, num_classes)

    def forward(self, x):
        out = self.relu1(self.linear1(x))
        out = self.relu2(self.linear2(out))
        out = self.relu3(self.linear3(out))
        out = self.classifier(out)       
        return out

net = Net().to(device)
print(net)

Net(
  (linear1): Linear(in_features=784, out_features=1024, bias=True)
  (relu1): ReLU()
  (linear2): Linear(in_features=1024, out_features=1024, bias=True)
  (relu2): ReLU()
  (linear3): Linear(in_features=1024, out_features=1024, bias=True)
  (relu3): ReLU()
  (classifier): Linear(in_features=1024, out_features=10, bias=True)
)


In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, weight_decay=1e-6, momentum=0.9)

In [9]:
print("\n---- Start Training ----")
best_accuracy = -1
for epoch in range(epochs):

    # TRAIN THE NETWORK
    train_loss, train_correct = 0, 0
    net.train()
    for inputs, targets in train_loader:
        # data is a list of [inputs, labels]
        inputs, targets = inputs.to(device), targets.to(device)
        ## care! net expect a 784 size vector and our dataset provide 1x28x28 = Reshape!
        inputs = inputs.view(inputs.size(0), -1)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        _, pred = outputs.max(1)  # get the index of the max log-probability
        train_correct += pred.eq(targets).sum().item()

        # print statistics
        train_loss += loss.item()
        
    train_loss /= len(train_loader.dataset)

    # TEST NETWORK
    net.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            ## care! net expect a 784 size vector and our dataset provide 1x28x28 = Reshape!
            inputs = inputs.view(inputs.size(0), -1)
            outputs = net(inputs)
            test_loss += criterion(outputs, targets)
            _, pred = outputs.max(1)  # get the index of the max log-probability
            correct += pred.eq(targets).sum().item()

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)
    print("[Epoch {}] Train Loss: {:.6f} - Test Loss: {:.6f} - Train Accuracy: {:.2f}% - Test Accuracy: {:.2f}%".format(epoch+1, train_loss, test_loss, 100. * train_correct / len(train_loader.dataset), test_accuracy))
    
    if test_accuracy>best_accuracy:
        best_accuracy = test_accuracy
    
print("Finished Training")
print("Best Test accuracy: {:.2f}".format(best_accuracy))


---- Start Training ----
[Epoch 1] Train Loss: 0.007333 - Test Loss: 0.002523 - Train Accuracy: 79.75% - Test Accuracy: 92.67%
[Epoch 2] Train Loss: 0.002037 - Test Loss: 0.001576 - Train Accuracy: 93.98% - Test Accuracy: 95.22%
[Epoch 3] Train Loss: 0.001308 - Test Loss: 0.001117 - Train Accuracy: 96.15% - Test Accuracy: 96.56%
[Epoch 4] Train Loss: 0.000948 - Test Loss: 0.000956 - Train Accuracy: 97.15% - Test Accuracy: 96.89%
[Epoch 5] Train Loss: 0.000725 - Test Loss: 0.000780 - Train Accuracy: 97.81% - Test Accuracy: 97.53%
[Epoch 6] Train Loss: 0.000553 - Test Loss: 0.000720 - Train Accuracy: 98.34% - Test Accuracy: 97.70%
[Epoch 7] Train Loss: 0.000444 - Test Loss: 0.000699 - Train Accuracy: 98.70% - Test Accuracy: 97.79%
[Epoch 8] Train Loss: 0.000346 - Test Loss: 0.000751 - Train Accuracy: 98.97% - Test Accuracy: 97.57%
[Epoch 9] Train Loss: 0.000264 - Test Loss: 0.000651 - Train Accuracy: 99.25% - Test Accuracy: 98.06%
[Epoch 10] Train Loss: 0.000203 - Test Loss: 0.000665 - 